# Step 2: Extract Marking Scheme to Excel

Use Gemini to parse the Word marking scheme into structured Excel sheets with error handling, validation, and progress tracking.

**Features:**
- ✅ Comprehensive error handling and validation
- ✅ Robust progress tracking and logging
- ✅ Improved markdown formatting and structure
- ✅ Robust file handling and backup
- ✅ Detailed validation and quality checks
- ✅ Professional output formatting

Configure the exam `prefix` and dataset folder in the next cell before running.


In [ ]:
from grading_utils import setup_paths, init_gemini_client
from google.genai import types
import mammoth
import html2text
from IPython.display import Markdown, display, clear_output
from pydantic import BaseModel, Field
from typing import List
import json
import pandas as pd
from termcolor import colored
import os
import shutil
import logging
from datetime import datetime

# Robust logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Robust Step 2: Extract Marking Scheme to Excel initialized")
print(f"✓ Session started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Configuration
prefix = "VTC Test"  # Change this to your exam name
dataset = "sample"   # Change to your dataset folder

# Setup paths with validation
try:
    paths = setup_paths(prefix, dataset)
    marking_scheme_word_file = f"../{dataset}/{prefix} Marking Scheme.docx"
    marking_scheme_excel_file = paths["marking_scheme_file"]
    
    # Validate input file exists
    if not os.path.exists(marking_scheme_word_file):
        raise FileNotFoundError(f"Marking scheme file not found: {marking_scheme_word_file}")
    
    logger.info(f"✓ Input file validated: {marking_scheme_word_file}")
    logger.info(f"✓ Output file will be: {marking_scheme_excel_file}")
    
    print(f"📁 Input: {marking_scheme_word_file}")
    print(f"📁 Output: {marking_scheme_excel_file}")
    
except Exception as e:
    logger.error(f"❌ Setup failed: {e}")
    raise

In [ ]:
# Robust Gemini client initialization with error handling
try:
    client = init_gemini_client()
    logger.info("✓ Gemini client initialized successfully")
    print("🤖 Gemini AI client ready")
except Exception as e:
    logger.error(f"❌ Failed to initialize Gemini client: {e}")
    print(f"❌ Gemini initialization failed: {e}")
    raise

In [ ]:
# Robust document processing with comprehensive error handling
def process_word_document(file_path):
    """Process Word document with error handling and validation"""
    try:
        logger.info(f"Processing Word document: {file_path}")
        
        # Validate file exists and is readable
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        
        file_size = os.path.getsize(file_path)
        logger.info(f"File size: {file_size:,} bytes")
        
        # Convert .docx to HTML using mammoth
        with open(file_path, "rb") as docx_file:
            result = mammoth.convert_to_html(docx_file)
            html_content = result.value
            
            # Check for conversion warnings
            if result.messages:
                logger.warning(f"Mammoth conversion warnings: {len(result.messages)}")
                for msg in result.messages[:5]:  # Show first 5 warnings
                    logger.warning(f"  - {msg}")
        
        # Convert HTML to markdown using html2text
        h = html2text.HTML2Text()
        h.ignore_links = False
        h.body_width = 0  # Don't wrap text
        h.ignore_images = True
        h.ignore_tables = False
        
        markdown_content = h.handle(html_content)
        
        # Validate content
        if not markdown_content.strip():
            raise ValueError("No content extracted from document")
        
        content_length = len(markdown_content)
        logger.info(f"✓ Extracted {content_length:,} characters of markdown content")
        
        return markdown_content, html_content
        
    except Exception as e:
        logger.error(f"❌ Document processing failed: {e}")
        raise

# Process the document
try:
    markdown_content, html_content = process_word_document(marking_scheme_word_file)
    print("✅ Document processed successfully")
    print(f"📄 Content length: {len(markdown_content):,} characters")
    
    # Display preview of markdown content
    print("\n📋 Document Preview:")
    display(Markdown(markdown_content[:1000] + "..." if len(markdown_content) > 1000 else markdown_content))
    
except Exception as e:
    print(f"❌ Document processing failed: {e}")
    raise


In [ ]:
# Robust Pydantic models with comprehensive validation
class Question(BaseModel):
    """Robust question model with comprehensive validation"""
    question_number: str = Field(
        description="The question number (e.g., '1', '2', '22a', '22b', 'Q1','Q2')"
    )
    question_text: str = Field(
        description="The full question text"
    )
    marking_scheme: str = Field(
        description="Well-formatted marking scheme using markdown. Use bullet points (-), numbered lists (1., 2.), bold (**text**) for key terms, and clear line breaks. Include point allocations in parentheses (e.g., '- Key concept explained (2 marks)'). Structure should be clear and scannable."
    )
    marks: int = Field(
        description="Total marks available for this question"
    )

class MarkingSchemeResponse(BaseModel):
    """Robust wrapper class with validation"""
    general_grading_guide: str = Field(
        default="",
        description="General grading guide for partial marks applicable to all questions, formatted in markdown"
    )
    questions: List[Question] = Field(
        description="List of questions with marking schemes and marks"
    )

logger.info("✓ Robust data models defined with validation")

In [ ]:
# Robust AI processing with comprehensive error handling and retry logic
def extract_marking_scheme_with_ai(markdown_content, max_retries=3):
    """Extract marking scheme using AI with error handling"""
    
    for attempt in range(max_retries):
        try:
            logger.info(f"AI extraction attempt {attempt + 1}/{max_retries}")
            
            # Create prompt
            prompt = f"""Please analyze this marking scheme document and extract structured, well-formatted data.

**FORMATTING REQUIREMENTS for marking_scheme:**
- Use markdown formatting (bullet points -, numbered lists 1., 2., bold **text**)
- Each marking criterion should be on its own line
- Show point allocations clearly (e.g., "- Correct formula (2 marks)")
- Use clear hierarchy with proper indentation for sub-points
- Add line breaks between major sections
- Bold important terms or key concepts
- Make it scannable and easy to read

**EXTRACT:**

1. **GENERAL GRADING GUIDE**: Extract any general grading guide or guidance for partial marks that applies to all/multiple questions (use markdown formatting)

2. **FOR EACH QUESTION**: Extract:
   - Question number (normalize to consistent format)
   - Question text (complete question statement)
   - **Marking scheme** (well-formatted with markdown, bullets, numbering, clear point allocation)
   - Total marks available (must be a positive integer)

**Important Guidelines:**
- When extracting the marking_scheme for each question, incorporate any general grading principles that apply to that question's scoring
- Ensure all questions have non-empty marking schemes
- Validate that mark totals are reasonable (1-100 marks per question)
- Use consistent formatting throughout

**Document Content:**

{markdown_content}
"""

            # Create configuration with structured output
            config = types.GenerateContentConfig(
                temperature=0.1,  # Lower temperature for more consistent extraction
                top_p=0.5,
                max_output_tokens=8192,  # Increased for larger documents
                response_mime_type="application/json",
                response_schema=MarkingSchemeResponse,
            )

            # Send to Gemini using structured output
            response = client.models.generate_content(
                model="gemini-3-pro-preview",
                contents=[{"role": "user", "parts": [{"text": prompt}]}],
                config=config,
            )

            # Robust response processing
            if hasattr(response, 'parsed') and response.parsed is not None:
                result = response.parsed
                general_guide = result.general_grading_guide
                questions_data = [q.model_dump() for q in result.questions]
                
                logger.info(f"✓ Successfully extracted {len(questions_data)} questions with structured output!")
                if general_guide:
                    logger.info(f"✓ General grading guide extracted ({len(general_guide)} characters)")
                
                return questions_data, general_guide
                
            else:
                # Robust fallback to text parsing
                response_text = response.text
                logger.warning("Structured output not available, attempting text parsing")
                
                try:
                    parsed_json = json.loads(response_text)
                    general_guide = parsed_json.get('general_grading_guide', '')
                    questions_data = parsed_json.get('questions', [])
                    
                    if not questions_data:
                        raise ValueError("No questions found in response")
                    
                    logger.info(f"✓ Successfully extracted {len(questions_data)} questions from text!")
                    return questions_data, general_guide
                    
                except json.JSONDecodeError as e:
                    logger.error(f"JSON parsing failed: {e}")
                    if attempt < max_retries - 1:
                        logger.info("Retrying with adjusted parameters...")
                        continue
                    else:
                        raise ValueError(f"Failed to parse AI response after {max_retries} attempts")
                        
        except Exception as e:
            logger.error(f"AI extraction attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                logger.info("Retrying...")
                continue
            else:
                raise

# Execute AI extraction
try:
    print("🤖 Processing document with Gemini AI...")
    questions_data, general_guide = extract_marking_scheme_with_ai(markdown_content)
    
    print(f"✅ AI extraction successful!")
    print(f"📊 Extracted {len(questions_data)} questions")
    if general_guide:
        print(f"📋 General grading guide: {len(general_guide)} characters")
    
except Exception as e:
    print(f"❌ AI extraction failed: {e}")
    raise


In [ ]:
# Robust validation and data processing
def validate_and_process_questions(questions_data, general_guide):
    """Comprehensive validation and processing of extracted questions"""
    try:
        logger.info("Validating extracted questions...")
        
        # Comprehensive validation
        validation_errors = []
        warnings = []
        
        if not questions_data:
            validation_errors.append("No questions extracted from document")
            
        for i, question in enumerate(questions_data):
            q_num = question.get('question_number', f'Question {i+1}')
            
            # Validate required fields
            if not question.get('question_text', '').strip():
                validation_errors.append(f"{q_num}: Missing question text")
            
            if not question.get('marking_scheme', '').strip():
                validation_errors.append(f"{q_num}: Missing marking scheme")
            
            # Validate marks
            marks = question.get('marks', 0)
            if not isinstance(marks, int) or marks <= 0:
                validation_errors.append(f"{q_num}: Invalid marks value ({marks})")
            elif marks > 50:
                warnings.append(f"{q_num}: High marks value ({marks}) - please verify")
        
        # Report validation results
        if validation_errors:
            print("\n" + "="*60)
            print(colored("❌ VALIDATION ERRORS DETECTED!", "red", attrs=['bold']))
            print("="*60)
            for error in validation_errors:
                print(colored(f"  • {error}", "red"))
            print("="*60)
            raise ValueError(f"Validation failed: {len(validation_errors)} error(s) found")
        
        if warnings:
            print("\n" + "⚠️  VALIDATION WARNINGS:")
            for warning in warnings:
                print(colored(f"  • {warning}", "yellow"))
            print()
        
        # Process questions - append general guide if available
        if general_guide and general_guide.strip():
            logger.info("Appending general grading guide to marking schemes")
            for question in questions_data:
                question['marking_scheme'] = f"{question['marking_scheme']}\n\n---\n\n**General Grading Guide:**\n{general_guide}"
        
        # Create DataFrame with formatting
        df = pd.DataFrame(questions_data)
        
        # Calculate statistics
        total_questions = len(questions_data)
        total_marks = df['marks'].sum()
        avg_marks = df['marks'].mean()
        
        logger.info(f"✓ Validation passed: {total_questions} questions, {total_marks} total marks")
        
        print(colored("✅ VALIDATION SUCCESSFUL!", "green", attrs=['bold']))
        print(f"📊 Questions: {total_questions}")
        print(f"📊 Total marks: {total_marks}")
        print(f"📊 Average marks per question: {avg_marks:.1f}")
        
        return df, total_questions, total_marks
        
    except Exception as e:
        logger.error(f"❌ Validation failed: {e}")
        raise

# Execute validation and processing
try:
    df, total_questions, total_marks = validate_and_process_questions(questions_data, general_guide)
    
    # Display results
    print("\n📋 Extracted Questions:")
    display(df[['question_number', 'question_text', 'marks']].head(10))
    
    if len(df) > 10:
        print(f"... and {len(df) - 10} more questions")
    
except Exception as e:
    print(f"❌ Validation failed: {e}")
    raise


In [ ]:
# Robust Excel export with comprehensive formatting and backupdef create_excel_report(df, marking_scheme_excel_file, total_questions, total_marks):    """Create comprehensive Excel report with multiple sheets and formatting"""    try:        logger.info(f"Creating Excel report: {marking_scheme_excel_file}")                # Create backup of existing file if it exists        if os.path.exists(marking_scheme_excel_file):            backup_file = f"{marking_scheme_excel_file}.backup.{datetime.now().strftime('%Y%m%d_%H%M%S')}"            shutil.copy2(marking_scheme_excel_file, backup_file)            logger.info(f"✓ Created backup: {backup_file}")                # Ensure output directory exists        os.makedirs(os.path.dirname(marking_scheme_excel_file), exist_ok=True)                # Create Excel writer with multiple sheets        with pd.ExcelWriter(marking_scheme_excel_file, engine='openpyxl') as writer:                        # Sheet 1: Marking Scheme (detailed rubric)            df.to_excel(writer, sheet_name='Marking Scheme', index=False)            logger.info("✓ Created 'Marking Scheme' sheet")                        # Sheet 2: Summary with statistics            summary_data = {                'Metric': [                    'Total Questions',                    'Total Marks',                    'Average Marks per Question',                    'Min Marks per Question',                    'Max Marks per Question',                    'Generated On',                    'Input File',                    'Output File'                ],                'Value': [                    total_questions,                    total_marks,                    f"{df['marks'].mean():.1f}",                    df['marks'].min(),                    df['marks'].max(),                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),                    os.path.basename(marking_scheme_word_file),                    os.path.basename(marking_scheme_excel_file)                ]            }            summary_df = pd.DataFrame(summary_data)            summary_df.to_excel(writer, sheet_name='Summary', index=False)            logger.info("✓ Created 'Summary' sheet")                        # Sheet 3: Question Overview (simplified view)            overview_df = df[['question_number', 'question_text', 'marks']].copy()            overview_df.to_excel(writer, sheet_name='Question Overview', index=False)            logger.info("✓ Created 'Question Overview' sheet")                        # Sheet 4: Validation Report            validation_data = {                'Check': [                    'All questions have marking schemes',                    'All questions have valid marks',                    'Question numbers are unique',                    'Total marks calculated',                    'General grading guide processed'                ],                'Status': [                    '✅ PASS' if all(q.get('marking_scheme', '').strip() for q in questions_data) else '❌ FAIL',                    '✅ PASS' if all(isinstance(q.get('marks', 0), int) and q.get('marks', 0) > 0 for q in questions_data) else '❌ FAIL',                    '✅ PASS' if len(set(q.get('question_number', '') for q in questions_data)) == len(questions_data) else '⚠️ WARNING',                    f'✅ PASS ({total_marks} marks)',                    '✅ PROCESSED' if general_guide else 'ℹ️ NOT FOUND'                ]            }            validation_df = pd.DataFrame(validation_data)            validation_df.to_excel(writer, sheet_name='Validation', index=False)            logger.info("✓ Created 'Validation' sheet")                # Verify file was created successfully        if os.path.exists(marking_scheme_excel_file):            file_size = os.path.getsize(marking_scheme_excel_file)            logger.info(f"✓ Excel file created successfully ({file_size:,} bytes)")            return True        else:            raise FileNotFoundError("Excel file was not created")                except Exception as e:        logger.error(f"❌ Excel export failed: {e}")        raise# Execute Excel exporttry:    success = create_excel_report(df, marking_scheme_excel_file, total_questions, total_marks)        if success:        print("\n" + "="*60)        print(colored("🎉 STEP 2 COMPLETED SUCCESSFULLY!", "green", attrs=['bold']))        print("="*60)        print(f"📁 Output file: {marking_scheme_excel_file}")        print(f"📊 Questions processed: {total_questions}")        print(f"📊 Total marks: {total_marks}")        print(f"📋 Sheets created: Marking Scheme, Summary, Question Overview, Validation")        print(f"⏰ Processing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")        print("="*60)        print("\n✅ Ready for Step 3: Question Annotations")        except Exception as e:    print(f"❌ Excel export failed: {e}")    raise
# Robust Excel export with comprehensive formatting and backup
def create_excel_report(df, marking_scheme_excel_file, total_questions, total_marks):
    """Create comprehensive Excel report with multiple sheets and formatting"""
    try:
        logger.info(f"Creating Excel report: {marking_scheme_excel_file}")
        
        # Create backup of existing file if it exists
        if os.path.exists(marking_scheme_excel_file):
            backup_file = f"{marking_scheme_excel_file}.backup.{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            shutil.copy2(marking_scheme_excel_file, backup_file)
            logger.info(f"✓ Created backup: {backup_file}")
        
        # Ensure output directory exists
        os.makedirs(os.path.dirname(marking_scheme_excel_file), exist_ok=True)
        
        # Create Excel writer with multiple sheets
        with pd.ExcelWriter(marking_scheme_excel_file, engine='openpyxl') as writer:
            # Sheet 1: Marking Scheme (detailed rubric)
            df.to_excel(writer, sheet_name='Marking Scheme', index=False)
            logger.info("✓ Created 'Marking Scheme' sheet")
            
            # Sheet 2: Summary with statistics
            summary_data = {
                'Metric': [
                    'Total Questions',
                    'Total Marks',
                    'Average Marks per Question',
                    'Min Marks per Question',
                    'Max Marks per Question',
                    'Generated On',
                    'Input File',
                    'Output File'
                ],
                'Value': [
                    total_questions,
                    total_marks,
                    f"{df['marks'].mean():.1f}",
                    df['marks'].min(),
                    df['marks'].max(),
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    os.path.basename(marking_scheme_word_file),
                    os.path.basename(marking_scheme_excel_file)
                ]
            }
            summary_df = pd.DataFrame(summary_data)
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
            logger.info("✓ Created 'Summary' sheet")
            
            # Sheet 3: Question Overview (simplified view)
            overview_df = df[['question_number', 'question_text', 'marks']].copy()
            overview_df.to_excel(writer, sheet_name='Question Overview', index=False)
            logger.info("✓ Created 'Question Overview' sheet")
            
            # Sheet 4: Validation Report
            validation_data = {
                'Check': [
                    'All questions have marking schemes',
                    'All questions have valid marks',
                    'Question numbers are unique',
                    'Total marks calculated',
                    'General grading guide processed'
                ],
                'Status': [
                    '✅ PASS' if all(q.get('marking_scheme', '').strip() for q in questions_data) else '❌ FAIL',
                    '✅ PASS' if all(isinstance(q.get('marks', 0), int) and q.get('marks', 0) > 0 for q in questions_data) else '❌ FAIL',
                    '✅ PASS' if len(set(q.get('question_number', '') for q in questions_data)) == len(questions_data) else '⚠️ WARNING',
                    f'✅ PASS ({total_marks} marks)',
                    '✅ PROCESSED' if general_guide else 'ℹ️ NOT FOUND'
                ]
            }
            validation_df = pd.DataFrame(validation_data)
            validation_df.to_excel(writer, sheet_name='Validation', index=False)
            logger.info("✓ Created 'Validation' sheet")
        
        # Verify file was created successfully
        if os.path.exists(marking_scheme_excel_file):
            file_size = os.path.getsize(marking_scheme_excel_file)
            logger.info(f"✓ Excel file created successfully ({file_size:,} bytes)")
            return True
        else:
            raise FileNotFoundError("Excel file was not created")
            
    except Exception as e:
        logger.error(f"❌ Excel export failed: {e}")
        raise

# Execute Excel export
try:
    success = create_excel_report(df, marking_scheme_excel_file, total_questions, total_marks)
    
    if success:
        print("\n" + "="*60)
        print(colored("🎉 STEP 2 COMPLETED SUCCESSFULLY!", "green", attrs=['bold']))
        print("="*60)
        print(f"📁 Output file: {marking_scheme_excel_file}")
        print(f"📊 Questions processed: {total_questions}")
        print(f"📊 Total marks: {total_marks}")
        print(f"📋 Sheets created: Marking Scheme, Summary, Question Overview, Validation")
        print(f"⏰ Processing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
        print("\n✅ Ready for Step 3: Question Annotations")
except Exception as e:
    print(f"❌ Excel export failed: {e}")
    raise